# Import and setup

In [15]:
from pylsl import StreamInfo, StreamInlet, StreamOutlet, resolve_stream

# Imports
import numpy as np
from tqdm import tqdm #loop progress bar
from scipy.io import loadmat #Loads in .mat File
import pandas as pd # To load in EEG data

import jupyter_beeper #Beep sound

import pyfirmata
import time

board = pyfirmata.Arduino('/dev/cu.usbmodem14301')
print('arduino connected!!')
for i in range(5):
    board.digital[13].write(1)
    time.sleep(1)
    board.digital[13].write(0)
    time.sleep(1)


arduino connected!!


# EMG data stream

In [21]:
eeg_id = 'openbcieeg12345'

print('Finding Stream...')
stream = resolve_stream('name', 'obci_eeg1')
inlet = StreamInlet(stream[0])
print('Found Stream')

Finding Stream...
Found Stream


## EMG data stream test

In [6]:
c,t = inlet.pull_chunk(timeout=2)
c = np.array(c).T[0]


0.13377675907462105
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected
movement detected


IndexError: index 0 is out of bounds for axis 0 with size 0

# Prototype Code

## Start prototype
jaw clentch command
> 2 clentch: activate
> 3 or more: deactivate

Other options:
> adjust strength based on strength of clentch 
    > (current/max_clentch) * 1
    > Max = 1
    > Min = 0

In [55]:
pin3 = board.get_pin('d:3:p')
# pin5 = board.get_pin('d:5:p')

pin5 = board.get_pin('d:5:p')
pin6 = board.get_pin('d:6:p')
pin9 = board.get_pin('d:9:p')
pin11 = board.get_pin('d:11:p')

In [ ]:
# c,t = inlet.pull_chunk(timeout=2)
# baseline = np.mean(np.array(c).T[0])
# print(baseline)
func_start = False
func_timeout = False
command_count = 0
time_thres = 1000
prev_time = 0
flex_thres = 0.3
count = 0
light = 0
b = jupyter_beeper.Beeper()
# 


# pin3.write(0.6)
while 1:
    samples, timestamp = inlet.pull_sample() # get EMG data sample and its timestamp

    curr_time = int(round(time.time() * 1000)) # get current time in milliseconds


    if ((samples[0] >=  flex_thres) & (curr_time - time_thres > prev_time)): # if an EMG spike is detected from the cheek muscles send 'G'
        prev_time = int(round(time.time() * 1000)) # update time
#         ser.write(b'Y') 
        command_count = command_count+1
        light = samples[0]
        b.beep(frequency=730, secs=0.1, blocking=True)
        if command_count == 1:
            pin5.write(1)
        elif command_count == 2:
            pin6.write(1)
        elif command_count == 3:
            pin9.write(1)
            
        print(command_count,light)
    # when timeout from jaw clentch command
    elif(curr_time - time_thres > prev_time): # if no spike is detected send 'B' 
        prev_time = int(round(time.time() * 1000)) # update time
        # 3 count mean on
        if command_count == 2:
            print('movement detected')
            pin3.write(1)
            print('R')
            b.beep(frequency=930, secs=0.1, blocking=True)
            pin11.write(1)
            time.sleep(1)
            pin5.write(0)
            pin6.write(0)
            pin9.write(0)
            pin11.write(0)


    #             board.digital[13].write(0)
    #             time.sleep(1)

        # 4 or more mean deactivate
        elif command_count > 2:
            board.digital[3].write(0)
            print('W')
            b.beep(frequency=930, secs=0.1, blocking=True)
            pin11.write(1)
            time.sleep(1)
            pin5.write(0)
            pin6.write(0)
            pin9.write(0)
            pin11.write(0)
            
        elif command_count == 1:
            b.beep(frequency=530, secs=0.1, blocking=True)
            pin11.write(1)
            time.sleep(1)
            pin5.write(0)
            pin6.write(0)
            pin9.write(0)
            pin11.write(0)


        #reset variables
        func_timeout = False
        command_count = 0
        func_start = False


1 0.386396199464798
2 0.38497194647789
3 0.9832583069801331
W
1 1.0
1 1.0
2 0.8149373531341553
3 0.7682231068611145
W
1 1.0
1 1.0
2 1.0
3 0.7600296139717102
4 1.0
5 0.8922858238220215
6 0.7935304045677185
W
1 0.6707481741905212
1 0.895986020565033
2 0.642082154750824
3 0.6960173845291138
W
1 1.0
1 0.9938427805900574
2 0.6295126676559448
movement detected
R
1 0.9995296597480774
2 0.9860287308692932
movement detected
R
1 0.38636675477027893
2 0.8188082575798035
3 0.9850791096687317
4 0.9027091264724731
W
1 1.0
2 0.5980629920959473
movement detected
R
1 0.7534964084625244
